##    NVD DATA: This notebook explains the process of parsing available NVD data from XML format to CSV, merging them to one master dataset, creating visulaization to gain initial insights.

### Parsing XML to CSV:

    We require few fields to be extraxted for our initial analysis, thus we will parse only those tags. There are files from 2002 to 2017 available on the NVD website(https://nvd.nist.gov/download.cfm). The code below only shows the procedure for the final year but the File names have to changed according to the file we are currently reading. This process has to be iterated. (Repeat this for all files individually from 2002 - 2017, including the file named recent)

In [ ]:
#Parsing using ElelmentTree
import xml.etree.ElementTree as ET
import csv

In [2]:
#Find root node
CVE_tree = ET.parse("nvdcve-2.0-2017.xml")
CVE_root= CVE_tree.getroot()

In [3]:
print (CVE_root)

<Element '{http://scap.nist.gov/schema/feed/vulnerability/2.0}nvd' at 0x49f1048>


In [4]:
#Create new CSV file to write the extracted fields
f = open('CVE_2017.csv', 'w')

In [5]:
#Extracting attributes and tags
CVE_count = 0;
CVE_listOfId = [];
for entry in CVE_root:
    cve_id = "";
    cwe_id = "";
    modified_date = "";
    cvss = "";
    for child in entry:
        
        #print (child.tag) #Print Child.tag will help you code further to identify child nodes
        
        if (child.tag == '{http://scap.nist.gov/schema/vulnerability/0.4}cve-id'):
            cve_id = child.text;
        if (child.tag == '{http://scap.nist.gov/schema/vulnerability/0.4}cwe'):
            cwe_id = child.attrib['id'];
        if (child.tag == '{http://scap.nist.gov/schema/vulnerability/0.4}cvss'):
            cvss = child.text;
        if (child.tag == '{http://scap.nist.gov/schema/vulnerability/0.4}published-datetime'):
            modified_date = child.text;
            
    #vuln = cve_id+","+cwe_id+","+modified_date+","+cvss+"\n";
    vuln = '{o1},{o2},{o3},{o1}\n'.format(o1=cve_id,o2=cwe_id,o3=modified_date,o4=cvss);
    f.write(vuln);
    CVE_count = CVE_count +1;

In [6]:
#This is to ensure that the file has been written into
print (CVE_count)
f.close();

484


## File Merge

Now that we have multiple individual CSV files corresponding to each year, we have to merge the files to yield one usable master database

In [7]:
fout=open("Merged_2002-17.csv","a")
# first file:
for line in open("CVE_2002.csv"):
    fout.write(line)
# now the rest:    
for num in range(2003,2017):
    f = open("CVE_"+str(num)+".csv")
    #f.next() # skip the header
    for line in f:
         fout.write(line)
    f.close() # not really needed
fout.close()

## Preparing dataset for visualization

To gain insights we need to know the number of vulnerabilites that have a valid CWE ID. We therefore create a new dataset that will make this visualization easy to produce.
The new dataset will contain the year, month and percentage ratio of number of CWE ID to CVE ID.

In [8]:
#using panda 
import pandas as pd
import csv

In [9]:
#Open merged file to calculate values
File= pd.read_csv("Merged_2002-17.csv")

In [10]:
#using hashmap, we will store a key value pair of every month and year combination
with open('Merged_2002-17.csv','r') as f:
    r = csv.reader(f, delimiter=',')
    cve_count = {};
    cwe_count = {};
    index = 0;
    for row in r:
        if(index!=0):
            year = row[2];
            month = row[3];
            CWE = row[1];
            CVE = row[0];
            timestamp = row[4];
            key = "01/"+month+"/"+year;
    #checking if that combination exists and incrementing count of CVE ID and CWE ID
            if key in cve_count:
                curr = cve_count[key];
                cve_count[key] = curr+1;
            else:
                cve_count[key] = 1;

            if CWE != "0":
                if key in cwe_count:
                    curr = cwe_count[key];
                    cwe_count[key] = curr+1;
                else:
                    cwe_count[key] = 1;
        index = index+1;

In [11]:
#creating a hashmap for the ratio values
ratioMap = {};
for k,v in cve_count.iteritems():
    cve_c = v;
    cwe_c = 0;
    if k in cwe_count:
        
        cwe_c = cwe_count[k];
    ratioMap[k] = round(100 * float(cwe_c)/float(cve_c),2);

In [12]:
#writing hashmap values to a file
f = open('ratio_v1.csv', 'w')
Header = "Date,Percentage\n";
f.write(Header);
for k,v in ratioMap.iteritems():
    
    outline = "{o1},{o3}\n".format(o1=k, o3=v);
    #print outline;
    f.write(outline);
f.close();

## Creating Visualization using Bokeh

In [13]:
from bokeh.charts import TimeSeries, show, output_file, vplot
import numpy as np
import pandas as pd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from dateutil import parser
from bokeh.layouts import column

In [20]:
#Open file for visualization
Plot_File= pd.read_csv("ratio_v1.csv")


In [21]:
date = Plot_File['Date'];
per = Plot_File['Percentage'];   

In [22]:
#at this point the csv doesn't take the date column to be a date but a string
i=0;
dt = [];
for d in date:
    dt.append(d)
    
    try:
        dt[i] = parser.parse(date[i]);
    except ValueError:
        print date[i];
    #No exceptions? We can proceed
    i+=1;

In [23]:
#create a dictionary
data = dict(
    Date=dt,
    PERCENTAGE=per)

In [33]:
#create a timeseries graph with points
tspoint = TimeSeries(data,
    x='Date', y=['PERCENTAGE'],
    color=['PERCENTAGE'], dash=['PERCENTAGE'], builder_type='point',
    title="Timeseries", ylabel='Stock Prices', legend=True)
#create a timeseries graph with a line
tsline = TimeSeries(data,
    x='Date', y=['PERCENTAGE'],
    color=['PERCENTAGE'], dash=['PERCENTAGE'], builder_type='line',
    title="Timeseries", ylabel='Stock Prices', legend=True)


In [35]:
#saves figures in an html file
output_file("timeseries_final.html")

In [36]:
#opens visualization
show(column(tsline,tspoint))